In [1]:
import pymarc as pymarc
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

class NeuralNet:
    
    def __init__(self, input=3, hidden=3, output=1):
        self.input = input
        self.hidden = hidden
        self.output = output
        
    def create_nodes(self):
        pass
            
        

class Node:
    
    def __init__(self):
        return None
          
    def _get_weights(self):
        return self.__weights
    
    def _set_weights(self, value):
        self.__weights = value
    
    weights = property(_get_weights, _set_weights)
   
    @property
    def bias(self):
        return self.__bias
    
class Layer:
    
    node_list = []
    
    def __init__(self, node_count):        
        self.node_count = node_count
        self.nodes = [Node() for node in range(self.node_count)]
    
    

    
    

        
        
def aggregate(values, weights, bias):
    return np.dot(values, weights) + bias * 1

def activate(aggregation):
    return 1 / (1 + np.exp(-aggregation))

def compute_total_error(target, output):
    return np.sum(1/2 * (target - output)**2)

def compute_e_partial_o1(target, output):
    return 2 * 1/2 * (target - output) * -1

def compute_o1_partial_net(output):
    return output * (1 - output)


In [ ]:
X = np.array([
    [-2,4,-1],
    [4,1,-1],
    [1, 6, -1],
    [2, 4, -1],
    [6, 2, -1],

])
weights = np.zeros(len(X[0]))
weights

In [ ]:
x = np.zeros((3, 8))
x.shape = (6, 4)
x

In [ ]:
for input in range(3):
    for hidden in range(3):
        print(input, "-->", hidden)

#### Input values
`input = np.array([0.05, 0.1])`

#### Input weights + bias
```input_weights_1 = np.array([0.15, 0.2])
input_weights_2 = np.array([0.25, 0.3])
bias_1 = np.array([0.35])```

#### Hidden layer weights + bias
```
hidden_weights_1 = np.array([.4, .45])
hidden_weights_2 = np.array([.5, .55])
bias_2 = np.array([.6])


h1 = Node(input, input_weights_1, bias_1, aggregate, activate)
h2 = Node(input, input_weights_2, bias_1, aggregate, activate)

h_out = np.concatenate([h1.activate(), h2.activate()])

o1 = Node(h_out, hidden_weights_1, bias_2, aggregate, activate)
o2 = Node(h_out, hidden_weights_2, bias_2, aggregate, activate)

o_out = np.concatenate([o1.activate(), o2.activate()])

e_o1 = compute_total_error(0.01, o1.activate())
e_o2 = compute_total_error(0.99, o2.activate())

e_total = e_o1 + e_o2

e_partial_o1 = compute_e_partial_o1(0.01, o1.activate())
o1_partial_net = compute_o1_partial_net(o1.activate())
o1_partial_w5 = 1 * h1.activate()
w5 = 0.4 - .5 * e_partial_o1 * o1_partial_net * o1_partial_w5

0.4 - 0.5 * 0.082167041
w5
```

In [65]:
#sample = np.array([[["Florian van Roekel", "Roekel, Florian van, 1980- photographer."], ["How Terry likes his coffee", "How Terry likes his coffee : a photo odyssey into office life /"]], [["Sun Dog Propaganda", "Sun Dog Propaganda,"], ["Banned in DC", "Banned in DC : photos and anecdotes from the DC punk underground (79-85) /"]]])
#for s in sample:
#    z = [fuzz.ratio(x[0], x[1])*0.01 for x in s]
#    print(z)

# Predict
def predict(s):
    prediction = []

    for p in s:
        if p > 0.7:
            prediction.append(1)
        if p < 0.5:
            prediction.append(0)
        if (p < 0.7) & (p > 0.5):
            prediction.append("?")
    return prediction

np.random.seed(1)

hidden_count = 2
output_count = 1

samples = np.array([[0.48, 0.58, 1], [0.97, 0.98, 1]])

bias = np.ones(hidden_count)
labels = np.array([[0], [1]])


# input --> hidden
#b_h = np.zeros(hidden_count)
w_h = np.random.normal(loc=0., scale=0.1, 
                       size=(len(samples[0]), hidden_count))

# hidden --> out
#b_out = np.zeros(output_count)
w_out = np.random.normal(loc=0., scale=0.1, 
                       size=(hidden_count, output_count)) 


for i in range(200):
    
    # input --> hidden
    #b_h = np.zeros(hidden_count)
    w_h = np.random.normal(loc=0., scale=0.1, 
                           size=(len(samples[0]), hidden_count))

    # hidden --> out
    #b_out = np.zeros(output_count)
    w_out = np.random.normal(loc=0., scale=0.1, 
                           size=(hidden_count, output_count)) 

    # step 1: net input of hidden layer
    # [n_samples, n_features] dot [n_features, n_hidden]
    # -> [n_samples, n_hidden]
    z_h = np.dot(samples, w_h)

    def activate(aggregation):
        return 1 / (1 + np.exp(-aggregation))

    # step 2: activation of hidden layer
    a_h = activate(z_h)

    # step 3: net input of output layer
    # [n_samples, n_hidden] dot [n_hidden, n_classlabels]
    # -> [n_samples, n_classlabels]
    z_out = np.dot(a_h, w_out) #+ b_out

    # step 4: activation output layer
    a_out = activate(z_out)

    
    print("---")
    print("Prediction: ", predict(a_out))

    # Cost 
    def error_tau_1(s, t):
        return np.sum(s[t == 0]**2)

    def error_tau_2(s, t):
        return np.sum((1 - s[t == 1])**2)

    Q = error_tau_1(a_out, labels) + error_tau_2(a_out, labels)

    def derive_Q_tau_1(s, t):
        return 2 * s[t == 0]

    def derive_Q_tau_2(s, t):
        return 2 * (1 - s[t == 1])

    def derive_sigma(s):
        return s * (1 - s)

    def derive_o_in(h):
        return h

    dQ_1 = derive_Q_tau_1(a_out, labels)
    dQ_2 = derive_Q_tau_2(a_out, labels)

    print("q1: ", dQ_1)
    print("q2: ", dQ_2)
    dw_out = dQ_1 * dQ_2 * derive_sigma(a_out) * derive_o_in(z_out)

    dw_h = dQ_1 * dQ_2 * derive_sigma(a_out) * w_out * derive_sigma(z_out) * samples

    w_out -= (0.1 * dw_out)

    w_h -= (0.1 * dw_h.T)

  



---
Prediction:  ['?', '?']
q1:  [ 1.00413828]
q2:  [ 0.99352889]
---
Prediction:  ['?', '?']
q1:  [ 1.03444993]
q2:  [ 0.96574922]
---
Prediction:  [0, 0]
q1:  [ 0.97288457]
q2:  [ 1.02710318]
---
Prediction:  ['?', '?']
q1:  [ 1.0560222]
q2:  [ 0.94560284]
---
Prediction:  ['?', '?']
q1:  [ 1.05689074]
q2:  [ 0.94256925]
---
Prediction:  ['?', '?']
q1:  [ 1.00949186]
q2:  [ 0.99043636]
---
Prediction:  ['?', '?']
q1:  [ 1.0044303]
q2:  [ 0.99552968]
---
Prediction:  [0, 0]
q1:  [ 0.94617515]
q2:  [ 1.05539329]
---
Prediction:  ['?', '?']
q1:  [ 1.02485627]
q2:  [ 0.97544619]
---
Prediction:  [0, 0]
q1:  [ 0.99262811]
q2:  [ 1.00731215]
---
Prediction:  ['?', '?']
q1:  [ 1.01272159]
q2:  [ 0.98693267]
---
Prediction:  ['?', '?']
q1:  [ 1.02565308]
q2:  [ 0.97433504]
---
Prediction:  [0, 0]
q1:  [ 0.97400561]
q2:  [ 1.0256999]
---
Prediction:  ['?', '?']
q1:  [ 1.03762348]
q2:  [ 0.96207149]
---
Prediction:  ['?', '?']
q1:  [ 1.04765108]
q2:  [ 0.95117173]
---
Prediction:  ['?', '?']
q

In [ ]:

                ##################
                # Backpropagation
                ##################

                # [n_samples, n_classlabels]
                sigma_out = a_out - y_train_enc[batch_idx]

                # [n_samples, n_hidden]
                sigmoid_derivative_h = a_h * (1. - a_h)

                # [n_samples, n_classlabels] dot [n_classlabels, n_hidden]
                # -> [n_samples, n_hidden]
                sigma_h = (np.dot(sigma_out, self.w_out.T) *
                           sigmoid_derivative_h)

                # [n_features, n_samples] dot [n_samples, n_hidden]
                # -> [n_features, n_hidden]
                grad_w_h = np.dot(X_train[batch_idx].T, sigma_h)
                grad_b_h = np.sum(sigma_h, axis=0)

                # [n_hidden, n_samples] dot [n_samples, n_classlabels]
                # -> [n_hidden, n_classlabels]
                grad_w_out = np.dot(a_h.T, sigma_out)
                grad_b_out = np.sum(sigma_out, axis=0)

                # Regularization and weight updates
                delta_w_h = (grad_w_h + self.l2*self.w_h)
                delta_b_h = grad_b_h # bias is not regularized
                self.w_h -= self.eta * delta_w_h
                self.b_h -= self.eta * delta_b_h

                delta_w_out = (grad_w_out + self.l2*self.w_out)
                delta_b_out = grad_b_out  # bias is not regularized
                self.w_out -= self.eta * delta_w_out
                self.b_out -= self.eta * delta_b_out

In [ ]:
# Activation of hidden layer
def activate(aggregation):
    return 1 / (1 + np.exp(-aggregation))
h_a = activate(h_in)

# Net input of output layer
# [n_samples, n_hidden] dot [n_hidden, n_classlabels]
# -> [n_samples, n_classlabels]
o_in = np.dot(h_a, output_weights)
o_a = activate(o_in)

# Cost function
def error_t1(s, p):
    p = p.reshape((2, 1))
    return np.sum(s[p == 0]**2)
        
def error_t2(s, p):
    p = p.reshape((2, 1))
    return np.sum((1 - s[p == 1])**2)

    
Q = np.sum(o_a**2) + np.sum((1 - o_a)**2)

error_t1(o_a, labels) + error_t2(o_a, labels)

# Predict
def predict(s):
    predictions = []    
    if s[s > 0.7]:
        predictions.append(1)
    if s[s < 0.5]:
        predictions.append(0)
    if s[(s < 0.7) & (s > 0.5)]:
        predictions.append("?")
    return predictions

# Backpropagation
# dQ/dw = dQ/ds * ds/dl * dl/dw
def derive_Q_tau_1(s, p):

    return 2 * s[p == 0]

def derive_Q_tau_2(s, p):

    return 2 * (1 - s[p == 1])

#derive_Q_tau_1(o_a, labels) * derive_Q_tau_2(o_a, labels)
def derive_sigma(s):
    return s * (1 - s)

def derive_o_in(h_a):
    return h_a

#dw = derive_Q_tau_1(o_a, labels) * derive_Q_tau_2(o_a, labels) * derive_sigma(o_a) * derive_o_in(h_a)

h_in

In [ ]:
#o_a[labels == 0].T
#np.dot(o_a[labels == 0], output_weights.T)
hid = np.random.randn(hidden_count, len(samples[0]))
out = np.random.randn(hidden_count, len(samples[0])) 

p = labels.reshape((2, 1))
o_a